# CDR Generator

In [23]:
import pandas as pd
import random
import math
from datetime import timedelta, datetime

## Loading contact list
Here a text file containing 100 contacts is loaded. The numbers are accorsing to Indian Phone Nuumbers Standards

# Functions to Generate a valid IMEI for Target Number

In [25]:
def luhn_residue(digits):
    return sum(sum(divmod(int(d)*(1 + i%2), 10))
                 for i, d in enumerate(digits[::-1])) % 10

In [26]:
def getImei(N):
    part = ''.join(str(random.randrange(0,9)) for _ in range(N-1))
    res = luhn_residue('{}{}'.format(part, 0))
    return '{}{}'.format(part, -res%10)

In [27]:
def get_Imei_list(number_of_imeis, number_of_logs):
    temp_list = []
    
    for i in range(number_of_imeis):
        temp_list.append(str(getImei(15)))
        
    IMEI_list = []
    for _ in range(number_of_logs):
        IMEI_list.append(random.choice(temp_list))
    
    return IMEI_list

# Function to generate Callers or Called List

In [28]:
def called_numbers(number_of_logs):
    call_list = []
    for i in range(number_of_logs):
        call_list.append(random.choice(list(contact_df['Contact Numbers'])))
    return call_list

# Generating Call Duration Logs

In [29]:
def random_start_call(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [30]:
def random_end_call(start):
    return start + timedelta(minutes=random.randrange(60), seconds=random.randrange(60))

In [31]:
#start_stamp = datetime.strptime('1/1/2020 1:30 PM', '%m/%d/%Y %I:%M %p')
#end_stamp = datetime.strptime('3/10/2020 4:50 AM', '%m/%d/%Y %I:%M %p')

def call_durations(cdr_start_stamp, cdr_end_stamp, number):
    call_durations={
        'call_start_stamp':[],
        'call_end_stamp':[],
    }
    
    start_stamp = datetime.strptime(cdr_start_stamp, '%m/%d/%Y %I:%M %p')
    end_stamp = datetime.strptime(cdr_end_stamp, '%m/%d/%Y %I:%M %p')
    
    for _ in range(number):
        call_durations['call_start_stamp'].append((random_start_call(start_stamp, end_stamp)))
        
    for i in call_durations['call_start_stamp']:
        call_durations['call_end_stamp'].append(random_end_call(i))
    
    return call_durations

# Generating Cell-ID for Analysis

In [32]:
def cid_generator(n, num=10):
    df = pd.read_excel('cell_id_chart.xlsx')
    df = df['MNC'].astype(str)+'-'+df['LAC'].astype(str) +'-'+df['Cell_id'].astype(str)
    cid_list = [random.choice(list(df)) for _ in range(num)]
    return [random.choice(cid_list) for _ in range(n)]

# Function to Generate CDR

In [33]:
def generate_cdr(IMEI_list, target_number_list, call_list, call_durations, cell_ids, n):
    
    cdr_df = pd.DataFrame({
        'IMEI_Number': IMEI_list,
        'Phone_Number': target_number_list,
        'Called_Numbers': call_list,
        'Type': [random.choice(['INCOMING','OUTGOING']) for i in range(n)],
        'Call_Start_Time': call_durations['call_start_stamp'],
        'Call_End_Time': call_durations['call_end_stamp'],
        'Cell ID': cell_ids
    })
    
#    cdr_df.to_csv('CDR for {}.csv'.format(target_number_list[0]))
#    cdr_df.to_json('CDR for {}.json'.format(target_number_list[0]))
    cdr_df.to_excel('CDR for {}.xlsx'.format(target_number_list[0]))
    print(cdr_df.head())

# Main Function

In [34]:
def main():
    
    target_number =  '+91-'+input("Enter the phone number(without country code): ")
    number_of_logs = int(input('Enter the number of call logs to generate: '))
    target_number_list = [str(target_number) for i in range(number_of_logs)]
    called_list = called_numbers(number_of_logs)
    number_of_imeis = int(input('Enter Number of random IMEI for {}: '.format(target_number)))
    imei_list = get_Imei_list(number_of_imeis, number_of_logs)
    
    cdr_start_stamp = input('''
    Enter the start date for the CDR in the format-
        (mm/dd/yyyy hh/mm AM/PM)
    =>      1/1/2020 1:30 PM
    
    Start Date: 
    ''')
    
    cdr_end_stamp = input('''
    Enter the end date for the CDR in the format-
        (mm/dd/yyyy hh/mm AM/PM)
    =>      1/1/2020 1:30 PM
    
    End Date: 
    ''')
    
    call_duration_dict=call_durations(cdr_start_stamp, cdr_end_stamp, number_of_logs)
    
    location_list = cid_generator(number_of_logs)
    generate_cdr(imei_list, target_number_list, called_list, call_duration_dict, location_list, number_of_logs)

In [43]:
contact_df = pd.read_fwf('file.txt', sep="\n", header=0, names = ['Contact Numbers'])
#contact_df.rename(columns = {0:'Contacts'}, inplace = True) 
contact_df.head()

,Contact Numbers
0,+91-985-550-6626
1,+91-985-556-8744
2,+91-935-551-4172
3,+91-855-524-2371
4,+91-855-579-5317


In [44]:
# Call this cell to start generating your CDR
main()

Enter the phone number(without country code): 7555720382
Enter the number of call logs to generate: 250
Enter Number of random IMEI for +91-7555720382: 3

    Enter the start date for the CDR in the format-
        (mm/dd/yyyy hh/mm AM/PM)
    =>      1/1/2020 1:30 PM
    
    Start Date: 
    1/1/2020 1:30 PM

    Enter the end date for the CDR in the format-
        (mm/dd/yyyy hh/mm AM/PM)
    =>      1/1/2020 1:30 PM
    
    End Date: 
    2/10/2020 12:00 AM
       IMEI_Number    Phone_Number    Called_Numbers      Type  \
0  653572001045655  +91-7555720382  +91-905-556-5719  OUTGOING   
1  525482228868465  +91-7555720382  +91-955-559-1994  INCOMING   
2  525482228868465  +91-7555720382  +91-755-565-8423  INCOMING   
3  653572001045655  +91-7555720382  +91-855-518-6589  INCOMING   
4  555008457115618  +91-7555720382  +91-855-579-5317  INCOMING   

      Call_Start_Time       Call_End_Time         Cell ID  
0 2020-01-18 08:08:10 2020-01-18 08:15:22    97-317-37172  
1 2020-01-05 18

# Summary
- This CDR generator is a reliable generator that produces CDR on User reuirements.
- CDR includes the following features:-
        - IMEI Number
        - Phone Number (Target Number)
        - Called/Caller Number
        - Call Start Stamp
        - Call End Stamp
        - Location of the Call(Lat & Long Coordinates)
- Locations are by default genrated on hard coded coordinates.

## Drawbacks
[ ] This generator seriously require an algorithm to produuce random location coordinates according to user needs keeping intact the analysis approach.

[ ] This Generator can not produce multiple CDRs (linked in some way with same contacts).

# OPEN FOR SUGGESTION AND IMPROVEMENTS.

![alt text](https://mtv.mtvnimages.com/uri/mgid:file:http:shared:mtv.com/news/wp-content/uploads/2015/12/suggestions-1449024387.gif?quality=.8&height=294&width=540)